# Αλγόριθμος ΚΝΝ

Στο παρακάτω παράδειγμα θα εξετάσουμε πως λειτουργεί ο αλγόριθμος Κ-Nearest Neighbors ([KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm?msclkid=0c75f966cf9c11ecbab5cba311a90428)), χρησιμοποιώντας το [iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set?msclkid=33057869cf9c11ec8e488a45734cbc68) σε προβλήματα κατηγοριοποίησης και προβλέψεων. Το αρχείο που θα χρησιμοποιήσετε είναι διαθέσιμο [**εδώ**](https://github.com/nkostopoulos/StochasticsLabPublic/blob/master/lab10/KNN/iris.csv).


### Ερωτήσεις



*   Να περιγράψετε συνοπτικά το τρόπο λειτουργίας του αλγορίθμου ΚΝΝ
*   Ποια είναι τα πλεονεκτήματα και τα μειονεκτήματα του συγκεκριμένου αλγορίθμου; 
*   Ο συγκεκριμένος αλγόριθμος είναι αποδοτικός στην περίπτωση που έχουμε μεγάλο πλήθος χαρακτηριστικών; Τι συμβαίνει στις περιπτώσεις όπου επιλέξουμε μεγάλο ή μικρό k; 



#### Eρώτηση 1:
Να περιγράψετε συνοπτικά το τρόπο λειτουργίας του αλγορίθμου ΚΝΝ



Ο τρόπος με τον οποίον λειτουργεί ο αλγόριθμος k-πλησιέστερων γειτόνων (kNN) είναι ο εξής:

- Αρχικά, ορίζεται μια τιμή $k \in \mathbb{N}$ ως υπερπαράμετρος από το σχεδιαστή ή χρήστη του αλγορίθμου και ο ταξινομητής τροφοδοτείται με και απομνημονεύει τα στιγμιότυπα του συνόλου εκπαίδευσης μαζί με την ετικέτα τους (target label)
- Ύστερα, για ένα νέο στιγμιότυπο, έστω $\vec{x}$, βρίσκουμε τα $k$ σε αριθμό πιο κοντινά σε αυτό στιγμιότυπα του συνόλου εκπαίδευσης $\vec{x}_1, \vec{x}_2,..., \vec{x}_k$. Για την εύρεση αυτών των διανυσμάτων, απαιτείται ο ορισμός της μετρικής απόστασης (πχ. ευκλείδια, manhattan, minkowsky, chebyshev κτλ) μεταξύ δύο στιγμιοτύπων $d(\vec{x},\vec{x}_i)$.
- Τέλος, παίρνοντας τα αντίστοιχα labels για τα κοντυνότερα διανύσματα $y_1, y_2, ..., y_k$, αναθέτουμε ως πρόβλεψη/ταξινόμηση κλάσης στο $\vec{x}$ την κλάση εκείνη που πλειοψηφεί στους κοντυνότερους γείτονες/διανύσματα.

Αναφέρουμε εδώ, πως ο ταξινομητής αυτός δεν εκπαιδεύεται όπως άλλοι ταξινομητές, αλλά απομνημονεύει όπως αναφέρθηκε τα δειγματικά στοιχεία εκπαίδευσης και έπειτα εκμεταλλευόμενος την τοπολογική εγγύτητα με άλλα στιγμιότυπα ταξινομεί νέα στιγμιότυπα.

In [24]:
# Make Predictions with k-nearest neighbors on the Iris Flowers Dataset
from csv import reader
from math import sqrt

In [25]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
		print('[%s] => %d' % (value, i))
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])



*   Αναφέρετε άλλες μεθόδους υπολογισμού της απόστασης



Στο παρακάτω κελί κώδικα, υλοποιείται η **ευκλείδια απόσταση** για δύο διανύσματα. Ωστόσο όπως αναφέρθηκε προηγουμένως, υπάρχουν και άλλες μετρικές αποστάσεις που μπορούν να χρησιμοποιηθούν αντ' αυτής.

- Euclidean distance: $d(\vec{x}, \vec{y}) = \sqrt{\sum_{i=1}^{m}|x_i-y_i|^2}$
- Manhattan distance: $d(\vec{x}, \vec{y}) = \sum_{i=1}^{m}|x_i-y_i|$
- Minkowsky distance: $d(\vec{x}, \vec{y}) = (\sum_{i=1}^{m} |x_i-y_i|^r)^{1/r} $
- Chebyshev distance: $d(\vec{x}, \vec{y}) = \max_{i=1,...,m}|x_i-y_i|$



In [26]:
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction



*   Δοκιμάστε με τα παρακάτω ζεύγη τιμών και σημειώστε τα αποτελέσματα


	*   [4.9, 3.1, 1.5, 0.1]
	*   [6.9, 3.1, 4.9, 1.5]
	*   [5.0, 2.0, 3.5, 1.0]
	*   [5.6, 2.7, 4.2, 1.3]
	*   [6.3, 3.3, 6.0, 2.5]
	*   [5.7, 2.9, 4.2, 1.3]
	*   [5.9, 3.0, 5.1, 1.8]











In [27]:
# Make a prediction with KNN on Iris Dataset
filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# define model parameter
num_neighbors = 5
# define a new record
row = [4.5, 2.3, 1.3, 0.3]
# predict the label
label = predict_classification(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

[Iris-virginica] => 0
[Iris-setosa] => 1
[Iris-versicolor] => 2
Data=[4.5, 2.3, 1.3, 0.3], Predicted: 1


Στο παρακάτω κελί φαίνονται τα αποτελέσματα του prediction για τα ζεύγη τιμών που ζητήθηκαν:

In [28]:
# define the new records for which to get predictions
rows = [[4.9, 3.1, 1.5, 0.1],
		[6.9, 3.1, 4.9, 1.5],
		[5.0, 2.0, 3.5, 1.0],
		[5.6, 2.7, 4.2, 1.3],
		[6.3, 3.3, 6.0, 2.5],
		[5.7, 2.9, 4.2, 1.3],
		[5.9, 3.0, 5.1, 1.8]]

for row in rows: # for each new sample point
	# predict the label
	label = predict_classification(dataset, row, num_neighbors)
	print('Data=%s, Distance=Euclidean, Predicted: %s' % (row, label))

Data=[4.9, 3.1, 1.5, 0.1], Distance=Euclidean, Predicted: 1
Data=[6.9, 3.1, 4.9, 1.5], Distance=Euclidean, Predicted: 2
Data=[5.0, 2.0, 3.5, 1.0], Distance=Euclidean, Predicted: 2
Data=[5.6, 2.7, 4.2, 1.3], Distance=Euclidean, Predicted: 2
Data=[6.3, 3.3, 6.0, 2.5], Distance=Euclidean, Predicted: 0
Data=[5.7, 2.9, 4.2, 1.3], Distance=Euclidean, Predicted: 2
Data=[5.9, 3.0, 5.1, 1.8], Distance=Euclidean, Predicted: 0




*   Δοκιμάστε να υπολογίσετε την απόσταση με τη μετρική [Manhattan](https://iq.opengenus.org/manhattan-distance/#:~:text=Manhattan%20distance%20is%20a%20distance%20metric%20between%20two,the%20measures%20in%20all%20dimensions%20of%20two%20points.?msclkid=50bbf70ecfa011ec91862d6b9263d761) για τα ζέυγη που σας έχουν δοθεί. Παρατηρείτε κάποια διαφοροποίηση ως προς τα αποτελέσματα;


In [29]:
# Calculate the Manhattan distance between two vectors
def manhattan_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += abs(row1[i] - row2[i])
	return distance

# Locate the most similar neighbors
def get_neighbors_manhattan(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = manhattan_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

# Make a prediction with neighbors
def predict_classification_manhattan(train, test_row, num_neighbors):
	neighbors = get_neighbors_manhattan(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

In [32]:
for row in rows: # for each new sample point
	# predict the label - this time with manhattan distance
	label = predict_classification_manhattan(dataset, row, num_neighbors)
	print('Data=%s, Distance=Manhattan, Predicted: %s' % (row, label))

Data=[4.9, 3.1, 1.5, 0.1], Distance=Manhattan, Predicted: 1
Data=[6.9, 3.1, 4.9, 1.5], Distance=Manhattan, Predicted: 2
Data=[5.0, 2.0, 3.5, 1.0], Distance=Manhattan, Predicted: 2
Data=[5.6, 2.7, 4.2, 1.3], Distance=Manhattan, Predicted: 2
Data=[6.3, 3.3, 6.0, 2.5], Distance=Manhattan, Predicted: 0
Data=[5.7, 2.9, 4.2, 1.3], Distance=Manhattan, Predicted: 2
Data=[5.9, 3.0, 5.1, 1.8], Distance=Manhattan, Predicted: 0


Δεν παρατηρούμε κάποια διαφορά στα αποτελέσματα, αλλά η ταξινόμηση είναι στις ίδιες κλάσεις για όλα τα δειγματικά στοιχεία.